In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import root_mean_squared_log_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
train_data = pd.read_csv("C:/Users/drmn_/Desktop/hw3-data/my_train.csv")
dev_data = pd.read_csv("C:/Users/drmn_/Desktop/hw3-data/my_dev.csv")
test_data = pd.read_csv("C:/Users/drmn_/Desktop/hw3-data/test.csv")

X_train = train_data.drop(["Id", "SalePrice"], axis=1)
y_train = train_data["SalePrice"]
X_dev = dev_data.drop(["Id", "SalePrice"], axis=1)
y_dev = dev_data["SalePrice"]

test_ids = test_data["Id"]
X_test = test_data.drop(["Id"], axis=1)

In [ ]:
y_train_log = np.log(y_train)
y_dev_log = np.log(y_dev) 

In [ ]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

In [ ]:
len(num_cols) + len(cat_cols)

In [ ]:
num_type_transformer = FunctionTransformer(lambda x: x.astype(float))
cat_type_transformer = FunctionTransformer(lambda x: x.astype(str))

In [ ]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("type_cast", cat_type_transformer), 
    ("onehot", OneHotEncoder(handle_unknown="ignore"))  
])

In [ ]:
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), 
    ("type_cast", num_type_transformer), 
    ("scaler", MinMaxScaler()),      
])


In [ ]:
poly_transformer = PolynomialFeatures(degree=2, interaction_only=False, include_bias=True)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, num_cols),  
        ("cat", categorical_transformer, cat_cols), 
    ])

In [ ]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),         
    ("polynomial", poly_transformer),      
    ("regressor", Ridge())                  
])

In [ ]:
param_grid = {
    "regressor__alpha": [0.01, 0.1, 1, 10, 100, 1000],
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_root_mean_squared_log_error")
grid_search.fit(X_train, y_train_log)

In [ ]:
print("Best alpha:", grid_search.best_params_)
print("Best score (negative MRSLE):", grid_search.best_score_)

dev_score = grid_search.score(X_dev, y_dev)
print("Test set score (negative MRSLE):", dev_score)

In [ ]:
predictions = np.exp(grid_search.best_estimator_.predict(X_test))
print("Predictions for new set (X_test):", predictions[:10]) 


In [ ]:
ridge_model = grid_search.best_estimator_.named_steps['regressor']
preprocessor = grid_search.best_estimator_.named_steps['preprocessor']
numerical_features_transformed = num_cols
categorical_features_transformed = preprocessor.named_transformers_["cat"].named_steps["onehot"].get_feature_names_out(cat_cols)
feature_names = np.concatenate([numerical_features_transformed, categorical_features_transformed])
weights = ridge_model.coef_
sorted_indices = np.argsort(weights)
most_negative_features = feature_names[sorted_indices[:10]]
most_positive_features = feature_names[sorted_indices[-10:]]
print("\nTop 10 most positive features:")

top_positive_features = sorted(zip(weights, feature_names), reverse=True)[:10]
top_negative_features = sorted(zip(weights, feature_names))[:10]

for coef, feature in top_positive_features:
    print(f"{feature}")

print("\nTop 10 most negative features:")
for coef, feature in top_negative_features:
    print(f"{feature}")

In [ ]:
train_predictions_log = grid_search.predict(X_train)
train_predictions = np.expm1(train_predictions_log)
train_rmsle = root_mean_squared_log_error(y_train, train_predictions)
print("Train RMSLE:", train_rmsle)
dev_predictions_log = grid_search.predict(X_dev) 
dev_predictions = np.expm1(dev_predictions_log)
dev_rmsle = root_mean_squared_log_error(y_dev, dev_predictions)
print("Dev RMSLE:", dev_rmsle)

In [ ]:
log_intercept = ridge_model.intercept_
print("Bias Feature Weight (intercept):", log_intercept)

In [ ]:
log_base_housing_price = ridge_model.intercept_
base_housing_price = np.exp(log_base_housing_price)
print("Log Base Housing Price:", log_base_housing_price)
print("Base Housing Price:", base_housing_price)

In [ ]:
test_pred_log = grid_search.predict(X_test)
test_pred = np.exp(test_pred_log)
ids = test_data['Id']
df_csv_file = pd.DataFrame({
    'Id': ids,
    'SalePrice': test_pred
})
df_csv_file.to_csv('Pred_HW3PR4_Polynomial.csv', index=False)
print("Pred_HW3PR4_Polynomial.csv file saved successfully")